**Notebook content:**
- First model: logistic regression 
- 

In [1]:
%reset -f 

In [2]:
jupyter nbconvert --to notebook --execute notebook.ipynb

SyntaxError: invalid syntax (<ipython-input-2-b0b5e9f151af>, line 1)

In [ ]:
from myDefs.defs import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn import metrics
from IPython.display import display

pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 200)
#pd.set_option('display.float_format', lambda x: '%.f' % x)
pd.options.display.precision = 4

#import sys
np.set_printoptions(threshold=sys.maxsize) #- print the full NumPy array

# visualization
import seaborn as sns
import matplotlib.pyplot as plt 
from matplotlib.pyplot import figure
%matplotlib inline


**Read events file**

In [ ]:
path = DATA_PATH + 'matrix1.xlsx'
df = pd.read_excel(path, header=[0,1,2,3], index_col=0)

In [ ]:
df.shape

In [ ]:
df.head()

### Fill missing data

In [ ]:
#fill numeric events with median 
df['numeric_events'] = df['numeric_events'].fillna(df['numeric_events'].median())

In [ ]:
#fill missing values in dtugs with 0 
df['drugs'] = df['drugs'].fillna(0)

In [ ]:
#fill numeric events with median 
df.iloc[:,1] = df.iloc[:,1].fillna(df.iloc[:,1].median())

In [ ]:
df.head()

In [ ]:
'''#read events
path = DATA_PATH + "parseData2.csv"
df = pd.read_csv(path, sep=',') 
#parse date of surgery
df['eventStartDate'] = pd.to_datetime(df['eventStartDate'], format='%Y-%m-%d')
df['eventEndDate'] = pd.to_datetime(df['eventEndDate'], format='%Y-%m-%d')

#read annonymous file
path = "{}parseAnonymous0.csv".format(DATA_PATH)
anonymous = pd.read_csv(path, sep=',')
#parse date of surgery
anonymous['Date of surgery'] = pd.to_datetime(anonymous['Date of surgery'], format='%Y-%m-%d')'''

In [ ]:
if WRITE_FLAG:
    df.to_excel(DATA_PATH + 'matrix_no_missing.xlsx')

# Logistic Regression 

### Get x and y

In [ ]:
to_drop = [df.columns[0], df.columns[2], df.columns[3], df.columns[4]]
x = df.drop(columns=to_drop)
x.head()

In [ ]:
y = df.iloc[:, 4]

### Exploration

In [ ]:
y.value_counts()

In [ ]:
sns.countplot(x=y).set(xlabel='Anastomotic Leak', ylabel='count')

In [ ]:
percent = y.sum()/len(y) 
print("percentage of num Anastomotic leak: {:.2f}%".format(percent*100))

### train - test

In [ ]:
#The stratify parameter makes a split so that the proportion of values in the sample produced 
#will be the same as the proportion of values provided to parameter stratify.

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=y)

traintest = pd.concat([y_train.value_counts(), y_test.value_counts()], axis=1)
traintest.columns = ['y_train', 'y_test']
traintest['total'] = traintest['y_train'] + traintest['y_test']
traintest

In [ ]:
model = LogisticRegression(solver='lbfgs', max_iter=100000)
model.fit(x_train, y_train)

### Performances

In [ ]:
y_pred = model.predict(x_test)
score = model.score(x_test, y_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(score))

In [ ]:
#precision is the ratio tp / (tp + fp)
"accuracy: {:.2f}".format((88+4)/(105))

In [ ]:
#plot confusion matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix
plot_confusion_matrix(model, x_test, y_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:

logit_roc_auc = metrics.roc_auc_score(y_test, y_pred)
fpr, tpr, thresholds = metrics.roc_curve(y_test, model.predict_proba(x_test)[:,1])

plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = {:.2f})'.format(logit_roc_auc))
plt.plot([0, 1], [0, 1],'r--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_train